In [1]:
import warnings
warnings.filterwarnings('ignore')

import pickle
import numpy as np
import scipy.sparse as sp
from scipy.sparse import load_npz

import torch

from cell.utils import link_prediction_performance
from cell.cell import Cell, EdgeOverlapCriterion, LinkPredictionCriterion
from cell.graph_statistics import compute_graph_statistics

# Load graph and validation-/ test edges (same split as in paper)

In [2]:
train_graph = load_npz('./data/CORA-ML_train.npz')
with open('./data/link_prediction.p', 'rb') as handle:
    val_ones, val_zeros, test_ones, test_zeros = pickle.load(handle)

## Edge overlap criterion

In [3]:
# initialize model with EO-criterion
model = Cell(A=train_graph,
             H=9,
             callbacks=[EdgeOverlapCriterion(invoke_every=10, edge_overlap_limit=.5)])

In [4]:
# train model 
model.train(steps=200,
            optimizer_fn=torch.optim.Adam,
            optimizer_args={'lr': 0.1,
                            'weight_decay': 1e-7})

Step:  10/200 Loss: 5.98659 Edge-Overlap: 0.042 Total-Time: 3
Step:  20/200 Loss: 4.05340 Edge-Overlap: 0.280 Total-Time: 7
Step:  30/200 Loss: 3.29106 Edge-Overlap: 0.451 Total-Time: 10
Step:  40/200 Loss: 2.94477 Edge-Overlap: 0.552 Total-Time: 14


In [5]:
generated_graph = model.sample_graph()

In [6]:
compute_graph_statistics(generated_graph)

{'d_max': 199.0,
 'd_min': 1.0,
 'd': 4.8277580071174375,
 'LCC': 2802,
 'wedge_count': 81516.0,
 'claw_count': 1781672.0,
 'triangle_count': 1428,
 'square_count': 6923.0,
 'power_law_exp': 1.8244785774294228,
 'gini': 0.4533693454905843,
 'rel_edge_distr_entropy': 0.9498521203506017,
 'assortativity': -0.07215054895022903,
 'clustering_coefficient': 0.0024044829800322393,
 'cpl': 5.288447214149108}

In [7]:
compute_graph_statistics(train_graph)

{'d_max': 238.0,
 'd_min': 1.0,
 'd': 4.8277580071174375,
 'LCC': 2810,
 'wedge_count': 101747.0,
 'claw_count': 3033514.0,
 'triangle_count': 2802,
 'square_count': 14268.0,
 'power_law_exp': 1.8550648593086239,
 'gini': 0.4825742921255409,
 'rel_edge_distr_entropy': 0.9406652031225717,
 'assortativity': -0.07626405450439543,
 'clustering_coefficient': 0.002771043746625201,
 'cpl': 5.630006245811316}

## Validation criterion

In [8]:
# initialize model with LP-criterion
model = Cell(A=train_graph,
             H=9,
             callbacks=[LinkPredictionCriterion(invoke_every=2,
                                                val_ones=val_ones,
                                                val_zeros=val_zeros,
                                                max_patience=3)])

In [9]:
# train model 
model.train(steps=200,
            optimizer_fn=torch.optim.Adam,
            optimizer_args={'lr': 0.1,
                            'weight_decay': 1e-6})

Step:   2/200 Loss: 7.92513 ROC-AUC Score: 0.604 Average Precision: 0.618 Total-Time: 0
Step:   4/200 Loss: 7.71120 ROC-AUC Score: 0.682 Average Precision: 0.706 Total-Time: 1
Step:   6/200 Loss: 7.26823 ROC-AUC Score: 0.728 Average Precision: 0.751 Total-Time: 1
Step:   8/200 Loss: 6.62998 ROC-AUC Score: 0.765 Average Precision: 0.789 Total-Time: 2
Step:  10/200 Loss: 5.93017 ROC-AUC Score: 0.804 Average Precision: 0.829 Total-Time: 2
Step:  12/200 Loss: 5.32614 ROC-AUC Score: 0.843 Average Precision: 0.864 Total-Time: 3
Step:  14/200 Loss: 4.88291 ROC-AUC Score: 0.873 Average Precision: 0.888 Total-Time: 3
Step:  16/200 Loss: 4.55538 ROC-AUC Score: 0.888 Average Precision: 0.901 Total-Time: 4
Step:  18/200 Loss: 4.26762 ROC-AUC Score: 0.897 Average Precision: 0.909 Total-Time: 4
Step:  20/200 Loss: 4.02134 ROC-AUC Score: 0.904 Average Precision: 0.914 Total-Time: 5
Step:  22/200 Loss: 3.83309 ROC-AUC Score: 0.912 Average Precision: 0.921 Total-Time: 5
Step:  24/200 Loss: 3.66701 ROC-

#### Link prediction performance for ROC-AUC score and average precision

In [10]:
link_prediction_performance(scores_matrix=model._scores_matrix, val_ones=test_ones, val_zeros=test_zeros)

(0.94483125, 0.9482979127234584)